In [21]:
import pandas as pd
from datetime import datetime

In [23]:
# Data import
df = pd.read_csv('/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/CRP/headlines_english_arabic_countries.csv', delimiter=',')

/var/folders/jn/gt3868vd039by14gxmgb54b40000gn/T/ipykernel_39282/2082097811.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/CRP/headlines_english_arabic_countries.csv', delimiter=',')


In [24]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773846 entries, 0 to 773845
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gkgrecordid          773846 non-null  object 
 1   date                 773846 non-null  float64
 2   subsourcecommonname  773743 non-null  object 
 3   documentidentifier   773846 non-null  object 
 4   enhancedthemes       693173 non-null  object 
 5   enhancedlocations    773846 non-null  object 
 6   tone                 773846 non-null  object 
 7   extrasxml            551861 non-null  object 
 8   translationinfo      0 non-null       float64
dtypes: float64(2), object(7)
memory usage: 53.1+ MB


,gkgrecordid,date,subsourcecommonname,documentidentifier,enhancedthemes,enhancedlocations,tone,extrasxml,translationinfo
0,20151011213000-456,2.015101e+13,dailynewsegypt.com,http://www.dailynewsegypt.com/2015/10/11/tgm-m...,"ECON_WORLDCURRENCIES_DOLLAR,2612;ECON_WORLDCUR...","4#Milan, Lombardia, Italy#IT#IT09#18363#45.466...","1.76991150442478,2.54424778761062,0.7743362831...",NaN,NaN
1,20151011213000-1551,2.015101e+13,gizmodo.com.au,http://www.gizmodo.com.au/2015/10/5-ideas-that...,"MANMADE_DISASTER_IMPLIED,58;MANMADE_DISASTER_I...","3#Manhattan, New York, United States#US#USNY#N...","0.843320017754106,2.13049267643142,1.287172658...",NaN,NaN
2,20151011213000-1644,2.015101e+13,competitor.com,http://velonews.competitor.com/2015/10/news/ch...,"WB_2670_JOBS,696;WB_2815_SKILLS_AND_EDUCATION,...","4#Abu Dhabi, Abu ZÂ¸Aby, United Arab Emirates#...","4.83271375464684,4.83271375464684,0,4.83271375...",NaN,NaN
3,20151016043000-1997,2.015102e+13,daily-chronicle.com,http://www.daily-chronicle.com/2015/10/15/saud...,"GENERAL_GOVERNMENT,620;GENERAL_GOVERNMENT,4524...","4#Baghdad, Baghdad, Iraq#IZ#IZ07#36785#33.3386...","-4.35754189944134,1.11731843575419,5.474860335...",NaN,NaN
4,20151011150000-503,2.015101e+13,dailynewsegypt.com,http://www.dailynewsegypt.com/2015/10/11/sharp...,"KIDNAP,2150;EXTREMISM,2184;BORDER,2772;TAX_REL...","4#Gaza, Israel (General), Israel#IS#IS00#18315...","-9.05730129390018,0.924214417744917,9.98151571...",NaN,NaN


# Preprocessing

In [25]:
# Date conversion
df.date = df.date.apply(lambda x: datetime.strptime(str(int(x)), '%Y%m%d%H%M%S'))
df.date

0        2015-10-11 21:30:00
1        2015-10-11 21:30:00
2        2015-10-11 21:30:00
3        2015-10-16 04:30:00
4        2015-10-11 15:00:00
                 ...        
773841   2023-03-07 13:45:00
773842   2023-03-07 19:15:00
773843   2023-03-07 19:15:00
773844   2023-03-07 19:15:00
773845   2023-03-07 19:15:00
Name: date, Length: 773846, dtype: datetime64[ns]

In [26]:
(df['tone'].str.count(',') + 1).value_counts()

7    773846
Name: tone, dtype: int64

In [34]:
#  Cleaning of the headlines
def themes_cleaning(s_):
    s_ = str(s_)
    modified_str = [elem.split(',')[0] for elem in s_.split(';')] # delete the number after the coma

    final_str=[]
    separator=' '
    for s in modified_str:
        if s.split('_')[0]=='WB':
            final_str.append(separator.join(s.split('_')[2:]).split(' ')) # delete the prefix 'WB_XXX', remove underscore and split each word
        else:
            final_str.append(separator.join(s.split('_')).split(' '))

    # to get a list of words and not a list of sub lists
    merged_list = []
    for sublist in final_str:
        merged_list.extend(sublist)
    
    merged_list = merged_list[:-1] if merged_list and merged_list[-1] == '' else merged_list

    return(merged_list) 

In [36]:
# Test of cleaning
original_str = 'WB_843_DISPUTE_RESOLUTION,1137;WB_843_DISPUTE_RESOLUTION,2805;TEST_TESTOS,2021'
res = themes_cleaning(original_str)
print(res, res[0])

['DISPUTE', 'RESOLUTION', 'DISPUTE', 'RESOLUTION', 'TEST', 'TESTOS'] DISPUTE


In [39]:
df['adj_enhancedthemes'] = df.enhancedthemes.apply(lambda x: themes_cleaning(x))

In [40]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    filtered_text = [word for word in text if word.lower() not in stop_words]
    return filtered_text

[nltk_data] Downloading package stopwords to /Users/flickr-
[nltk_data]     xc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
df['adj_enhancedthemes'] = df['adj_enhancedthemes'].apply(lambda x: remove_stopwords(x))

In [43]:
df.adj_enhancedthemes.iloc[7]

['ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'ECON',
 'TAXATION',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'USPEC',
 'POLICY1',
 'TAX',
 'FNCACT',
 'MINISTER',
 'TAX',
 'FNCACT',
 'MINISTER',
 'TAX',
 'FNCACT',
 'MINISTER',
 'TAX',
 'FNCACT',
 'MINISTER',
 'TAX',
 'FNCACT',
 'MINISTER']

In [44]:
#  Cleaning of the locations
def locations_cleaning(s_):
    s_ = str(s_)
    modified_str = [elem.split('#')[1] for elem in s_.split(';')]
    final_str = [elem.split(', ') for elem in modified_str ]
    merged_list = []
    for sublist in final_str:
        merged_list.append(sublist[-1]) # we only keep the titles of country
    return(merged_list) 


In [45]:
# Test of cleaning
print(df.enhancedlocations.iloc[8])
res = locations_cleaning(df.enhancedlocations.iloc[8])
print(res)

1#Indonesian#ID#ID##-5#120#ID#432;1#Australian#AS#AS##-27#133#AS#973;4#Sydney, New South Wales, Australia#AS#AS02#154637#-33.8833#151.217#-1603135#2342;1#Australia#AS#AS##-27#133#AS#238;1#Australia#AS#AS##-27#133#AS#331;1#Australia#AS#AS##-27#133#AS#1787;1#Australia#AS#AS##-27#133#AS#2353;1#Oman#MU#MU##21#57#MU#1038;1#United Arab Emirates#AE#AE##24#54#AE#1077;1#Kuwait#KU#KU##29.3375#47.6581#KU#1970;3#Hollywood, California, United States#US#USCA#CA037#34.0983#-118.327#1660757#1656
['Indonesian', 'Australian', 'Australia', 'Australia', 'Australia', 'Australia', 'Australia', 'Oman', 'United Arab Emirates', 'Kuwait', 'United States']


In [46]:
df['adj_enhancedlocations'] = df.enhancedlocations.apply(lambda x: locations_cleaning(x))

In [48]:
# Drop empty headlines
df = df.dropna(subset=['enhancedthemes'])
df = df.dropna(subset=['subsourcecommonname'])
df.isnull().sum()

gkgrecordid                   0
date                          0
subsourcecommonname           0
documentidentifier            0
enhancedthemes                0
enhancedlocations             0
tone                          0
extrasxml                206398
translationinfo          693073
adj_enhancedthemes            0
adj_enhancedlocations         0
dtype: int64

In [49]:
# add three columns which specify the proportion of occurrence of Egypt, UAE and KSA
def calculate_proportion(names,criteria):
    return sum(1 for name in names if any(criterion in name.lower() for criterion in criteria) ) / len(names)

# Add a new column to the DataFrame with the proportion of 'China' or 'Chinese'
df['Egypt Proportion'] = df.adj_enhancedlocations.apply(lambda x : calculate_proportion(x,['egypt','egyptian']))
df['UAE Proportion'] = df.adj_enhancedlocations.apply(lambda x : calculate_proportion(x,['united arab emirates','uae']))
df['KSA Proportion'] = df.adj_enhancedlocations.apply(lambda x : calculate_proportion(x,['ksa','saudi arabia','king of saudi arabia']))



In [50]:
df.iloc[:,-3:]

,Egypt Proportion,UAE Proportion,KSA Proportion
0,0.863636,0.045455,0.000000
1,0.000000,0.013699,0.000000
2,0.000000,1.000000,0.000000
3,0.000000,0.000000,0.111111
4,0.555556,0.000000,0.000000
...,...,...,...
773840,0.000000,1.000000,0.000000
773841,0.000000,0.000000,0.088235
773842,0.000000,0.666667,0.000000
773844,0.000000,0.000000,0.178571


# Split the Dataset

In [51]:
# Filter the news whose enhancedthemes contain UAE/KSA/EGYPT respectively 
# and whose proportion of occurrence among all the mentioned locations should be higher than 30%

condition_egypt = (df['Egypt Proportion'].apply(lambda x: x > 0.3) ) & \
            (df['adj_enhancedthemes'].apply(lambda x: any(keyword in x for keyword in ['EGYPT','EGYPTIAN'])))
df_EGYPT = df[condition_egypt]

condition_uae = (df['UAE Proportion'].apply(lambda x: x > 0.3) ) & \
            (df['adj_enhancedthemes'].apply(lambda x: any(keyword in x for keyword in ['UAE','ARAB','ERAMITES'])))
df_UAE = df[condition_uae]

condition_ksa = (df['KSA Proportion'].apply(lambda x: x > 0.3) ) & \
            (df['adj_enhancedthemes'].apply(lambda x: any(keyword in x for keyword in ['KSA','SAUDI','ARABIA'])))
df_KSA = df[condition_ksa]


In [52]:
# print(df_UAE.date.describe())
df_UAE.info()
df_UAE.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71740 entries, 2 to 773842
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   gkgrecordid            71740 non-null  object        
 1   date                   71740 non-null  datetime64[ns]
 2   subsourcecommonname    71740 non-null  object        
 3   documentidentifier     71740 non-null  object        
 4   enhancedthemes         71740 non-null  object        
 5   enhancedlocations      71740 non-null  object        
 6   tone                   71740 non-null  object        
 7   extrasxml              52130 non-null  object        
 8   translationinfo        0 non-null      float64       
 9   adj_enhancedthemes     71740 non-null  object        
 10  adj_enhancedlocations  71740 non-null  object        
 11  Egypt Proportion       71740 non-null  float64       
 12  UAE Proportion         71740 non-null  float64       
 13  

,gkgrecordid,date,subsourcecommonname,documentidentifier,enhancedthemes,enhancedlocations,tone,extrasxml,translationinfo,adj_enhancedthemes,adj_enhancedlocations,Egypt Proportion,UAE Proportion,KSA Proportion
2,20151011213000-1644,2015-10-11 21:30:00,competitor.com,http://velonews.competitor.com/2015/10/news/ch...,"WB_2670_JOBS,696;WB_2815_SKILLS_AND_EDUCATION,...","4#Abu Dhabi, Abu ZÂ¸Aby, United Arab Emirates#...","4.83271375464684,4.83271375464684,0,4.83271375...",NaN,NaN,"[JOBS, SKILLS, EDUCATION, JOBS, STRATEGIES, TR...",[United Arab Emirates],0.0,1.000000,0.0
5,20151011150000-539,2015-10-11 15:00:00,yahoo.com,https://en-maktoob.news.yahoo.com/mideast-stoc...,"ECON_OILPRICE,114;ECON_STOCKMARKET,52;ECON_STO...","4#First Gulf Bank, Dubayy, United Arab Emirate...","0.465116279069767,0.930232558139535,0.46511627...",NaN,NaN,"[ECON, OILPRICE, ECON, STOCKMARKET, ECON, STOC...","[United Arab Emirates, Qatar, Qatar, United Ar...",0.0,0.600000,0.0
25,20151014104500-0,2015-10-14 10:45:00,khaleejtimes.com,http://khaleejtimes.com/nation/crime/man-murde...,"TAX_FNCACT_WOMAN,101;TAX_FNCACT_WOMAN,916;TAX_...","4#Jebel Ali, Dubayy, United Arab Emirates#AE#A...","-7.12589073634204,0.475059382422803,7.60095011...",NaN,NaN,"[TAX, FNCACT, WOMAN, TAX, FNCACT, WOMAN, TAX, ...","[United Arab Emirates, United Arab Emirates, Q...",0.0,0.533333,0.0
30,20151018111500-661,2015-10-18 11:15:00,zawya.com,http://www.zawya.com/story/HMHs_Ajman_Palace_H...,"TAX_FNCACT_ASSISTANT,861;TAX_FNCACT_DIRECTOR,8...","4#Ajman, ÂAjman, United Arab Emirates#AE#AE02...","9.58904109589041,9.58904109589041,0,9.58904109...",NaN,NaN,"[TAX, FNCACT, ASSISTANT, TAX, FNCACT, DIRECTOR...","[United Arab Emirates, United Arab Emirates, U...",0.0,1.000000,0.0
31,20151018111500-724,2015-10-18 11:15:00,dubaicityguide.com,http://www.dubaicityguide.com/site/news/news-d...,"SOC_EMERGINGTECH,1781;WB_1467_EDUCATION_FOR_AL...","4#Dubai, Dubayy, United Arab Emirates#AE#AE03#...","3.93873085339168,4.37636761487965,0.4376367614...",NaN,NaN,"[SOC, EMERGINGTECH, EDUCATION, EDUCATION, EDUC...",[United Arab Emirates],0.0,1.000000,0.0


In [53]:
df_UAE.to_csv('df_UAE.csv',index=False)
df_KSA.to_csv('df_KSA.csv',index=False)
df_EGYPT.to_csv('df_EGYPT.csv',index=False)
